In [ ]:
from sae_utils import (
    clear_gpu_cache,
    load_sae_from_dir,
    load_gemma_2_sae,
)

device = "cuda:4"
sae_path = "/data2/xzwnlp/gemma-scope-9b-pt-res/layer_24/width_16k/average_l0_114"
if "gemma" in sae_path.lower():
    sae, sparsity = load_gemma_2_sae(sae_path, device=device)
else:
    sae, sparsity = load_sae_from_dir(sae_path, device=device)


/data2/xzwnlp/anaconda3/envs/sae/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [1]:
import requests
neuronpedia_id="gemma-2-9b/24-gemmascope-res-16k"
query="dog"

modelId, saeId = neuronpedia_id.split('/')
url = "https://www.neuronpedia.org/api/explanation/search"
payload = {
    "modelId": modelId,
    "layers": [saeId],
    "query": query
}
headers = {"Content-Type": "application/json"}

response = requests.post(url, json=payload, headers=headers)

print(response.json())

{'request': {'modelId': 'gemma-2-9b', 'layers': ['24-gemmascope-res-16k'], 'query': 'dog'}, 'results': [{'modelId': 'gemma-2-9b', 'layer': '24-gemmascope-res-16k', 'index': '10502', 'description': 'references to dog behavior and training contexts', 'explanationModelName': 'gpt-4o-mini', 'typeName': 'oai_token-act-pair', 'cosine_similarity': 0.4702212196720381, 'neuron': {'modelId': 'gemma-2-9b', 'layer': '24-gemmascope-res-16k', 'index': '10502', 'sourceSetName': 'gemmascope-res-16k', 'creatorId': 'cljj57d3c000076ei38vwnv35', 'createdAt': '2024-08-29T11:27:55.783Z', 'maxActApprox': 64.612, 'hasVector': False, 'vector': [], 'vectorLabel': None, 'vectorDefaultSteerStrength': 10, 'hookName': None, 'topkCosSimIndices': [10502, 9010, 9001, 8558, 8278, 14153, 13911, 12523, 14822, 12802, 2576, 381, 8500, 13395, 13511, 5657, 10937, 5009, 9781, 3744, 11583, 8602, 12759, 15298, 1130], 'topkCosSimValues': [1, 0.3364, 0.3191, 0.1993, 0.1987, 0.1967, 0.1959, 0.1813, 0.1759, 0.1654, 0.1602, 0.1316, 

In [10]:
import torch
sae_feature_vector = torch.zeros_like(sae.W_dec[0]).to(sae.device)
sae_feature_vector += sae.W_dec[0]
sae_feature_vector

tensor([ 0.0040,  0.0061,  0.0094,  ..., -0.0033,  0.0013, -0.0060],
       device='cuda:4', grad_fn=<AddBackward0>)

In [ ]:
sae_feature_vector.cpu()

tensor([ 0.0040,  0.0061,  0.0094,  ..., -0.0033,  0.0013, -0.0060],
       grad_fn=<ToCopyBackward0>)

In [80]:
def get_feature_description(modelId, saeId, position_ids):
    import requests
    import pandas as pd

    url = f"https://www.neuronpedia.org/api/explanation/export?modelId={modelId}&saeId={saeId}"
    headers = {"Content-Type": "application/json"}
    response = requests.get(url, headers=headers)

    data = response.json()
    description_df = pd.DataFrame(data)
    # rename index to "feature"
    description_df.rename(columns={"index": "feature_id"}, inplace=True)
    description_df["feature_id"] = description_df["feature_id"].astype(int)
    description_df["description"] = description_df["description"].apply(
        lambda x: x.lower()
    )
    result_dict = {k: description_df[description_df["feature_id"] == k].to_dict(orient='records') for k in position_ids}
    
    for k in position_ids:
        for item in result_dict[k]:
            print(f"Feature ID: {item['feature_id']}, Description: {item['description']}, ExplanationModelName: {item['explanationModelName']}")

    return result_dict

position_ids = [0,1,2]
modelId = "gemma-2-9b"
saeId = "24-gemmascope-res-16k"

get_feature_description(modelId, saeId, position_ids)

Feature ID: 0, Description:  structured data or code blocks in programming-related documents, ExplanationModelName: None
Feature ID: 1, Description:  elements and functions related to building and managing software components, specifically in programming contexts, ExplanationModelName: None
Feature ID: 2, Description:  java data structure imports and declarations, ExplanationModelName: None


{0: [{'modelId': 'gemma-2-9b',
   'layer': '24-gemmascope-res-16k',
   'feature_id': 0,
   'description': ' structured data or code blocks in programming-related documents',
   'explanationModelName': None,
   'typeName': 'oai_token-act-pair'}],
 1: [{'modelId': 'gemma-2-9b',
   'layer': '24-gemmascope-res-16k',
   'feature_id': 1,
   'description': ' elements and functions related to building and managing software components, specifically in programming contexts',
   'explanationModelName': None,
   'typeName': 'oai_token-act-pair'}],
 2: [{'modelId': 'gemma-2-9b',
   'layer': '24-gemmascope-res-16k',
   'feature_id': 2,
   'description': ' java data structure imports and declarations',
   'explanationModelName': None,
   'typeName': 'oai_token-act-pair'}]}